In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import os
import sys
from pathlib import Path

# Добавляем путь на уровень выше
sys.path.append(str(Path(os.getcwd()).resolve().parent))

from utils.features import *
from utils.load_data import load_all_data
from models.dfdgcn import DFDGCN

In [67]:
data_dir = Path('../data/PEMS-BAY')
metadata, data, adj = load_all_data(data_dir)

In [6]:
import os
import pickle
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
from node2vec import Node2Vec
from sklearn.manifold import TSNE
from sklearn.cluster  import KMeans


In [13]:
import numpy as np
import networkx as nx
from node2vec import Node2Vec

G = nx.from_numpy_array(adj)
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, p=1, q=1, workers=12)
model = node2vec.fit(window=10, min_count=1, batch_words=4)
node_embeddings = np.array([model.wv[str(node)] for node in G.nodes()])

Computing transition probabilities:   0%|          | 0/325 [00:00<?, ?it/s]

In [14]:
node_embeddings_expanded = np.expand_dims(node_embeddings, axis=0)  # Форма: (1, 325, 64)
node_embeddings_expanded = np.repeat(node_embeddings_expanded, data.shape[0], axis=0)  # Форма: (2016, 325, 64)

# 4. Объединение с временными рядами по последней оси
data = np.concatenate([data, node_embeddings_expanded], axis=-1)  # Форма: (2016, 325, 3 + 64)

# 5. Проверка результата
print("Исходные данные (временные ряды):", data.shape)
print("Эмбеддинги узлов:", node_embeddings.shape)
print("Объединённые данные:", data.shape)

Исходные данные (временные ряды): (2016, 325, 67)
Эмбеддинги узлов: (325, 64)
Объединённые данные: (2016, 325, 67)


In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from ..models.dfdgcn import DFDGCN
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

# Данные и параметры
L, N, C = data.shape  # [2016, 325, C]
batch_size = 16
train_ratio = 0.7
val_ratio = 0.1
test_ratio = 0.2
seq_len = 12  # Количество временных шагов на вход
pred_len = 12  # Количество временных шагов для предсказания

# Индексы каналов для нормализации
normalize = True

channels_to_normalize = [0] #+ [i for i in range(3, 64)]

# Проверка корректности разделения данных
assert train_ratio + val_ratio + test_ratio == 1.0, "Сумма долей train, val и test должна быть равна 1.0"

# Разделение данных на train, val и test
num_samples = data.shape[0]  # Количество временных шагов (L)
train_size = int(num_samples * train_ratio)
val_size = int(num_samples * val_ratio)
test_size = num_samples - train_size - val_size

train_data = data[:train_size, :, :]  # [train_size, N, C]
val_data = data[train_size:train_size + val_size, :, :]  # [val_size, N, C]
test_data = data[train_size + val_size:, :, :]  # [test_size, N, C]

# Убедитесь, что данные имеют разрешение на запись
train_data = train_data.copy()
val_data = val_data.copy()
test_data = test_data.copy()

# Нормализация данных
if normalize:
    assert all(0 <= ch < C for ch in channels_to_normalize), "Индексы каналов выходят за пределы допустимого диапазона"
    channel_max = train_data[:, :, channels_to_normalize].max(axis=(0, 1), keepdims=True)  # Форма [1, 1, len(channels_to_normalize)]
    channel_max[channel_max == 0] = 1.0
    train_data[:, :, channels_to_normalize] = train_data[:, :, channels_to_normalize] / channel_max
    val_data[:, :, channels_to_normalize] = val_data[:, :, channels_to_normalize] / channel_max
    test_data[:, :, channels_to_normalize] = test_data[:, :, channels_to_normalize] / channel_max


# Создание кастомного Dataset
class TrafficDataset(Dataset):
    def __init__(self, data, seq_len, pred_len):
        super().__init__()
        self.data = data  # Форма [L, N, C]
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        # Количество возможных последовательностей
        return self.data.shape[0] - self.seq_len - self.pred_len + 1

    def __getitem__(self, idx):
        # Извлекаем последовательность входных данных
        x = self.data[idx:idx + self.seq_len, :, :]  # Форма [seq_len, N, C]
        # Извлекаем целевую последовательность
        y = self.data[idx + self.seq_len:idx + self.seq_len + self.pred_len, :, 0]  # Форма [pred_len, N, C]
        return x, y

# Создание DataLoader
train_dataset = TrafficDataset(train_data, seq_len, pred_len)
val_dataset = TrafficDataset(val_data, seq_len, pred_len)
test_dataset = TrafficDataset(test_data, seq_len, pred_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [71]:
for x, y in train_loader:
    print(x.shape, y.shape)
    1/0

torch.Size([16, 12, 325, 3]) torch.Size([16, 12, 325])


ZeroDivisionError: division by zero

In [77]:
# Определение устройства
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'
print(device)

# Обновление модели, данных и вычислений
supports = [torch.tensor(adj, dtype=torch.float32)]
model = DFDGCN(num_nodes=N, supports=supports, in_dim=C, out_dim=pred_len).to(device)
criterion = nn.MSELoss()  # Функция потерь
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

cuda


In [78]:
from torch.utils.tensorboard import SummaryWriter
import time
# Инициализация TensorBoard
writer = SummaryWriter(log_dir=f"runs/original/")

def compute_metrics(output, target):
    """Вычисление метрик MAE, RMSE, MAPE."""
    abs_error = torch.abs(output - target).sum().item()
    mae = abs_error / len(target)
    rmse = ((output - target) ** 2).sum().item() / len(target)
    rmse = rmse ** 0.5
    mape = (abs_error / torch.abs(target).sum().item()) if torch.abs(target).sum().item() != 0 else 0
    return mae, rmse, mape

def train_val_test_model(model, train_loader, val_loader, test_loader, epochs):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        # === Тренировка ===
        model.train()
        train_loss, train_mae, train_rmse, train_mape = 0.0, 0.0, 0.0, 0.0
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} - Training", leave=False)

        for x, y in train_loader_tqdm:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x).squeeze(-1)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x.size(0)

            mae, rmse, mape = compute_metrics(output, y)
            train_mae += mae
            train_rmse += rmse
            train_mape += mape

        train_loss /= len(train_loader.dataset)
        train_mae /= len(train_loader)
        train_rmse /= len(train_loader)
        train_mape /= len(train_loader)

        writer.add_scalar("Loss/Train", train_loss, epoch + 1)
        writer.add_scalar("MAE/Train", train_mae, epoch + 1)
        writer.add_scalar("RMSE/Train", train_rmse, epoch + 1)
        writer.add_scalar("MAPE/Train", train_mape, epoch + 1)

        # === Валидация ===
        model.eval()
        val_loss, val_mae, val_rmse, val_mape = 0.0, 0.0, 0.0, 0.0
        val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{epochs} - Validation", leave=False)

        with torch.no_grad():
            for x, y in val_loader_tqdm:
                x, y = x.to(device), y.to(device)
                output = model(x).squeeze(-1)
                loss = criterion(output, y)
                val_loss += loss.item() * x.size(0)

                mae, rmse, mape = compute_metrics(output, y)
                val_mae += mae
                val_rmse += rmse
                val_mape += mape

        val_loss /= len(val_loader.dataset)
        val_mae /= len(val_loader)
        val_rmse /= len(val_loader)
        val_mape /= len(val_loader)

        writer.add_scalar("Loss/Validation", val_loss, epoch + 1)
        writer.add_scalar("MAE/Validation", val_mae, epoch + 1)
        writer.add_scalar("RMSE/Validation", val_rmse, epoch + 1)
        writer.add_scalar("MAPE/Validation", val_mape, epoch + 1)

        # === Тестирование ===
        test_loss, test_mae, test_rmse, test_mape = 0.0, 0.0, 0.0, 0.0
        test_loader_tqdm = tqdm(test_loader, desc=f"Epoch {epoch + 1}/{epochs} - Testing", leave=False)

        with torch.no_grad():
            for x, y in test_loader_tqdm:
                x, y = x.to(device), y.to(device)
                output = model(x).squeeze(-1)
                loss = criterion(output, y)
                test_loss += loss.item() * x.size(0)

                mae, rmse, mape = compute_metrics(output, y)
                test_mae += mae
                test_rmse += rmse
                test_mape += mape

        test_loss /= len(test_loader.dataset)
        test_mae /= len(test_loader)
        test_rmse /= len(test_loader)
        test_mape /= len(test_loader)

        writer.add_scalar("Loss/Test", test_loss, epoch + 1)
        writer.add_scalar("MAE/Test", test_mae, epoch + 1)
        writer.add_scalar("RMSE/Test", test_rmse, epoch + 1)
        writer.add_scalar("MAPE/Test", test_mape, epoch + 1)

        # Сохранение лучшей модели
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")

    writer.close()


In [79]:
train_val_test_model(model, train_loader, val_loader, test_loader, epochs=50)